# Extracción de Series de Indicadores del BCRP

#### Este script permite extraer series de datos del Banco Central de Reserva del Perú (BCRP) en formato CSV mediante web scraping.

In [ ]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

# Diccionario de conversión de meses
MESES = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}

def convertir_fecha(fecha: str) -> str:
    '''Convierte fechas del formato 'Mes.Año' al formato '01/mm/aaaa'.'''
    mes, anio = fecha.split('.')
    return f'01/{MESES.get(mes, '01')}/{anio}'

def obtener_indicador_bcrp(codigo: str, nombre_columna: str, inicio: str, fin: str) -> pd.DataFrame:
    '''
    Extrae una serie del BCRP API y devuelve un DataFrame.
    
    Parámetros:
        codigo: Código de la serie (ej. 'PN01770AM').
        nombre_columna: Nombre que tendrá la columna de valores.
        inicio: Fecha de inicio (ej. '2018-01').
        fin: Fecha de fin (ej. '2023-12').

    Retorna:
        DataFrame con columnas ['Fecha', nombre_columna]
    '''
    url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/csv/{inicio}/{fin}'
    response = requests.get(url)

    if response.status_code != 200:
        print(f'Error al descargar {codigo}: {response.status_code}')
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    texto = soup.get_text()

    pattern = r'"(.*?)","(.*?)"'
    matches = re.findall(pattern, texto)

    data = [{'Fecha': convertir_fecha(fecha), nombre_columna: float(valor)} for fecha, valor in matches]
    return pd.DataFrame(data)

def combinar_series(indicadores: list, inicio: str, fin: str) -> pd.DataFrame:
    '''
    Combina múltiples indicadores en un solo DataFrame.

    Parámetros:
        indicadores: Lista de tuplas (nombre_columna, codigo_bcrp)
        inicio: Fecha inicial (ej. '2018-01')
        fin: Fecha final (ej. '2023-12')
    
    Retorna:
        DataFrame combinado.
    '''
    df_final = None

    for nombre, codigo in indicadores:
        df = obtener_indicador_bcrp(codigo, nombre, inicio, fin)
        if df is not None:
            if df_final is None:
                df_final = df
            else:
                df_final = df_final.merge(df, on='Fecha', how='outer')
    
    return df_final


Ajuste la lista de indicadores y los periodos que usted quiera

In [ ]:
# Lista de indicadores: (nombre_columna, código_bcrp)
indicadores = [
    ('Nombre1', 'Código1'),
    ('Nombre3', "Código3"),
]

# Parámetros de período
inicio = '2018-01'
fin = '2023-12'

# Obtener DataFrame combinado
df_completo = combinar_series(indicadores, inicio, fin)

# Guardar CSV
df_completo.to_csv('data/BCRP_data.csv', index=False)
print('Datos guardados en 'data/BCRP_data.csv'')

Datos guardados en 'data/BCRP_Indicadores.csv'
